In [2]:
import pandas as pd
df=pd.read_csv("books_cleaned.csv")
df.shape

(5197, 13)

In [3]:
df.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [4]:
pd.set_option("display.max_rows", None)  # show all rows
pd.set_option("display.max_columns", None)

df["categories"].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [5]:
df[df["categories"]=="non-fiction"]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description


In [6]:
category_mapping = {
    'Fiction': 'Fiction',
    'Juvenile Fiction': "Children's Fiction",
    'Biography & Autobiography': 'Nonfiction',
    'History': 'Nonfiction',
    'Literary Criticism': 'Nonfiction',
    'Philosophy': 'Nonfiction',
    'Religion': 'Nonfiction',
    'Comics & Graphic Novels': 'Fiction',
    'Drama': 'Fiction',
    'Juvenile Nonfiction': "Children's Nonfiction",
    'Science': 'Nonfiction',
    'Poetry': 'Fiction'
}
df['simple_category'] = df['categories'].map(category_mapping)


In [7]:
df[~df['simple_category'].isna()].shape

(3743, 14)

In [8]:
df.shape

(5197, 14)

In [9]:
import torch
import random
import numpy as np

# ✅ Fix all randomness
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


In [10]:
# Use a pipeline as a high-level helper
from transformers import pipeline
fiction_categories=["fiction","Nonfiction"]
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli",
                device="cpu")

Device set to use cpu


In [11]:
pipe.model.eval()
def generate_predictions(sequence, candidate_labels):
    result = pipe(sequence, candidate_labels)          # run pipeline once
    scores = np.array(result["scores"])                  # ensure array-like
    max_index = np.argmax(scores)
    max_label = result["labels"][max_index]
    return max_label

In [12]:
def generate_predictions(sequence, candidate_labels):
    result = pipe(sequence, candidate_labels)
    # ✅ Always pick the label with the highest score
    return result["labels"][np.argmax(result["scores"])]


In [13]:
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# fiction_df = df.loc[df["simple_category"] == "Fiction"].reset_index(drop=True)

# actual_cats = []
# predicted_cats = []

# for i in tqdm(range(min(300, len(fiction_df)))):
#     sequence = fiction_df.loc[i, "description"]
#     predicted_label = generate_predictions(sequence, fiction_categories)
#     predicted_cats.append(predicted_label)
#     actual_cats.append("Fiction")


In [14]:
# predictions_df = pd.DataFrame({
#     "actual_categories": actual_cats,
#     "predicted_categories": predicted_cats
# })

# predictions_df["correct_prediction"] = np.where(
#     predictions_df["actual_categories"].str.lower() == predictions_df["predicted_categories"].str.lower(),
#     1,
#     0
# )

# accuracy = predictions_df["correct_prediction"].mean() * 100
# print(f"✅ Model Accuracy: {accuracy:.2f}%")


In [15]:
sequence=df.loc[df["simple_category"]=="Fiction","description"].reset_index(drop=True)[0]                                                                                                                                                                                                         

In [16]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [17]:
import numpy as np
max_index = np.argmax(pipe(sequence, fiction_categories)['scores'])
max_label = pipe(sequence, fiction_categories)['labels'][max_index]
max_label

'fiction'

In [18]:
def generate_predictions(sequence, categories):
    predictions = pipe(sequence, categories)
    max_index = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_index]
    return max_label

In [24]:
import numpy as np
import pandas as pd
from tqdm import tqdm

actual_cats = []
predicted_cats = []

fiction_categories = ["fiction", "nonfiction"]  # or use full-sentence labels for better accuracy

# --- Fiction samples ---
fiction_df = df.loc[df["simple_category"] == "Fiction"].reset_index(drop=True)

for i in tqdm(range(min(300, len(fiction_df))), desc="Predicting Fiction"):
    sequence = fiction_df.loc[i, "description"]
    predicted_label = generate_predictions(sequence, fiction_categories)
    predicted_cats.append(predicted_label)
    actual_cats.append("Fiction")

# --- Nonfiction samples ---
nonfiction_df = df.loc[df["simple_category"] == "Nonfiction"].reset_index(drop=True)

for i in tqdm(range(min(300, len(nonfiction_df))), desc="Predicting Nonfiction"):
    sequence = nonfiction_df.loc[i, "description"]
    predicted_label = generate_predictions(sequence, fiction_categories)
    predicted_cats.append(predicted_label)
    actual_cats.append("Nonfiction")

# --- Combine results into a DataFrame ---
predictions_df = pd.DataFrame({
    "actual_categories": actual_cats,
    "predicted_categories": predicted_cats
})

# --- Mark correct predictions ---
predictions_df["correct_prediction"] = np.where(
    predictions_df["actual_categories"].str.lower() == predictions_df["predicted_categories"].str.lower(),
    1,
    0
)

# --- Calculate accuracy ---
accuracy = predictions_df["correct_prediction"].mean() * 100
print(f"✅ Model Accuracy: {accuracy:.2f}%")



Predicting Fiction:   0%|          | 0/300 [00:00<?, ?it/s]

Predicting Nonfiction: 100%|██████████| 300/300 [07:27<00:00,  1.49s/it]

✅ Model Accuracy: 68.50%


In [25]:

predictions_df

,actual_categories,predicted_categories,correct_prediction
0,Fiction,fiction,1
1,Fiction,nonfiction,0
2,Fiction,nonfiction,0
3,Fiction,nonfiction,0
4,Fiction,fiction,1
5,Fiction,nonfiction,0
6,Fiction,fiction,1
7,Fiction,fiction,1
8,Fiction,fiction,1
9,Fiction,nonfiction,0


In [ ]:
# Count total and correct predictions per actual category
category_stats = (
    predictions_df.groupby("actual_categories")["correct_prediction"]
    .agg(["count", "sum"])
    .rename(columns={"count": "total", "sum": "correct"})
)

# Calculate accuracy %
category_stats["accuracy (%)"] = (category_stats["correct"] / category_stats["total"]) * 100

print(category_stats)


                   total  correct  accuracy (%)
actual_categories                              
Fiction              300      131     43.666667
Nonfiction           300      280     93.333333


In [29]:
isbns = []
predicted_cats = []

missing_cats = df.loc[df["simple_category"].isna(), ["isbn13", "description"]].reset_index(drop=True)

for i in tqdm(range(len(missing_cats))):
    sequence = missing_cats.loc[i, "description"]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    isbns += [missing_cats.loc[i, "isbn13"]]


 21%|██▏       | 311/1454 [06:56<25:29,  1.34s/it]  


KeyboardInterrupt: 

In [27]:

missing_predicted_df = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_cats})

NameError: name 'isbns' is not defined

In [28]:
missing_predicted_df

NameError: name 'missing_predicted_df' is not defined

In [ ]:
df= pd.merge(df, missing_predicted_df, on="isbn13", how="left")
df["simple_category"] = np.where(df["simple_category"].isna(), df["predicted_categories"], df["simple_category"])
df = df.drop(columns = ["predicted_categories"])

NameError: name 'pd' is not defined

In [ ]:
df[df["categories"].str.lower().isin([
    "romance",
    "science fiction",
    "scifi",
    "fantasy",
    "horror",
    "mystery",
    "thriller",
    "comedy",
    "crime",
    "historical"
])]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_category
24,9780006513087,0006513085,Gravity,Tess Gerritsen,Science fiction,http://books.google.com/books/content?id=KI66c...,Emma Watson a research physician has been trai...,2004.0,4.04,342.0,8024.0,Gravity,9780006513087 Emma Watson a research physician...,nonfiction
475,9780099410355,0099410354,Traitor,Matthew Woodring Stover,Science fiction,http://books.google.com/books/content?id=VbICO...,"From the depths of catastrophe, a glimmer of h...",2002.0,4.00,320.0,6765.0,Traitor,"9780099410355 From the depths of catastrophe, ...",nonfiction
478,9780099422341,0099422344,Yeats is Dead!,Joseph O'Connor,Comedy,http://books.google.com/books/content?id=DrE3I...,"In aid of Amnesty International, this is a bri...",2002.0,3.39,298.0,34.0,Yeats is Dead!: A Novel by Fifteen Irish Writers,"9780099422341 In aid of Amnesty International,...",fiction
491,9780099446729,0099446723,Blackwood Farm,Anne Rice,Horror,http://books.google.com/books/content?id=cIn8T...,"Lestat Is Back, Saviour And Demon, Presiding O...",2003.0,3.86,774.0,26145.0,Blackwood Farm,"9780099446729 Lestat Is Back, Saviour And Demo...",fiction
1090,9780261102422,0261102427,The Silmarillion,John Ronald Reuel Tolkien,Fantasy,http://books.google.com/books/content?id=22ePu...,Tolkien's Silmarillion is the core work of the...,1999.0,3.91,384.0,253.0,The Silmarillion,9780261102422 Tolkien's Silmarillion is the co...,fiction
1435,9780340837955,0340837950,Stranger in a Strange Land,Robert A. Heinlein,Science fiction,http://books.google.com/books/content?id=ZQhiP...,"Epic, entertaining, Stranger in a Strange Land...",2005.0,3.92,672.0,563.0,Stranger in a Strange Land,"9780340837955 Epic, entertaining, Stranger in ...",fiction
1439,9780345251220,0345251229,Visions from Nowhere,William Arrow,Science fiction,NaN,"The first novel in the series, ""Return to the ...",1976.0,3.23,183.0,10.0,Visions from Nowhere,"9780345251220 The first novel in the series, ""...",fiction
2845,9780575075597,0575075597,Replay,Ken Grimwood,Fantasy,http://books.google.com/books/content?id=9vmNP...,At forty-three Jeff Winston is tired of his lo...,2005.0,4.16,272.0,412.0,Replay,9780575075597 At forty-three Jeff Winston is t...,nonfiction
2860,9780590254762,0590254766,"The lion, the witch and the wardrobe",Clive Staples Lewis,Fantasy,NaN,Four English school children enter the magic l...,1995.0,4.21,189.0,860.0,"The lion, the witch and the wardrobe",9780590254762 Four English school children ent...,nonfiction
3288,9780739423851,0739423851,Wizard's Castle,Diana Wynne Jones,Fantasy,http://books.google.com/books/content?id=hB7hA...,Howl's moving castle - Eldest of three sisters...,2002.0,4.44,376.0,439.0,Wizard's Castle,9780739423851 Howl's moving castle - Eldest of...,nonfiction


In [ ]:
df[df["simple_category"].isna()].shape

NameError: name 'df' is not defined

In [ ]:
# df.to_csv("books_with_categories1.csv", index=False)